# **Retail Sales Analysis modules**

## Objectives

* Create a set of modules to support the analysis of the Retail Sales Data set from Kaggle to act as demonstration of skills

## Inputs

* Data will be pulled using the Kaggle API.

## Outputs

* Multiple python modules that can be loaded and resused to support re-use of analytics

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

Hard code working directory to relative root directory of workbook.

In [4]:
import os
os.chdir('/workspace/RetailSalesRework')
print("You set a new current directory")

You set a new current directory


In [2]:
## Modules and Extensions imported
import pandas as pd
import os
from datetime import timedelta, date
import kagglehub

# Data Acquisition

Load data using Kagglehub api - the user will need a Kaggle account and a kaggle api code.  Both of the can be obtained from [Kaggle.com](https://Kaggle.com/).  This project will be using the [Retail Data Anlaytics DataSet](https://www.kaggle.com/datasets/manjeetsingh/retaildataset)

In [3]:
## Setup Kagglehub and login
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
## load Retail Data Set using Kagglehub
kagglehub.dataset_download("manjeetsingh/retaildataset")

'/home/gitpod/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2'

In [5]:
## List of files in folder
os.listdir('/home/gitpod/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2/')


['Features data set.csv', 'sales data-set.csv', 'stores data-set.csv']

In [6]:
## Load data sets
Features_df = pd.read_csv('/home/gitpod/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2/Features data set.csv')
Stores_df = pd.read_csv('/home/gitpod/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2/stores data-set.csv')
Sales_df = pd.read_csv('/home/gitpod/.cache/kagglehub/datasets/manjeetsingh/retaildataset/versions/2/sales data-set.csv')

In [14]:
## Merge sales_df and Stores_df on Store id - change data type of Date to Date, Store to Object and Dept to Object,
Sales_df = pd.merge(Sales_df, Stores_df, how='left', on='Store')
Sales_df['Date'] = pd.to_datetime(Sales_df['Date'], format='%d/%m/%Y')
Sales_df['Store'] = Sales_df['Store'].astype(object)
Sales_df['Dept'] = Sales_df['Dept'].astype(object)
Sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421570 entries, 0 to 421569
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  object        
 1   Dept          421570 non-null  object        
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Type          421570 non-null  object        
 6   Size          421570 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 22.9+ MB


In [16]:
"""
This approach did not work efficiently  so has been commented out

## create module which calculates an aggregated function for a specified column for a stated date range.

## for a specified type. holiday status 

def calculate_rolling_mean_dynamic(row, Sales_df, window=13):
    
    Calculate the rolling mean dynamically for each row based on its Type and IsHoliday.
    
    Parameters:
        row (pd.Series): A single row of the DataFrame.
        Sales_df (pd.DataFrame): The sales DataFrame containing 'Date', 'Type', 'IsHoliday', and 'Weekly_Sales'.
        window (int): The rolling window size (default: 13 weeks).
    
    Returns:
        float: The rolling mean for the given row.
    
    # Filter the DataFrame for the current row's Type and IsHoliday
    filtered_df = Sales_df[
        (Sales_df['Type'] == row['Type']) & 
        (Sales_df['IsHoliday'] == row['IsHoliday'])
    ].sort_values(by='Date')
    
    # Ensure Date is datetime and set as index
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df = filtered_df.set_index('Date')
    
    # Calculate rolling mean
    filtered_df['Rolling_Mean'] = (
        filtered_df['Weekly_Sales']
        .rolling(window=window, min_periods=1)
        .mean()
    )
    
    # Return the rolling mean for the current row's date
    return filtered_df.loc[row['Date'], 'Rolling_Mean']

def calculate_rolling_stdev_dynamic(row, Sales_df, window=13):
    
    Calculate the rolling mean dynamically for each row based on its Type and IsHoliday.
    
    Parameters:
        row (pd.Series): A single row of the DataFrame.
        Sales_df (pd.DataFrame): The sales DataFrame containing 'Date', 'Type', 'IsHoliday', and 'Weekly_Sales'.
        window (int): The rolling window size (default: 13 weeks).
    
    Returns:
        float: The rolling mean for the given row.
    
    # Filter the DataFrame for the current row's Type and IsHoliday
    filtered_df = Sales_df[
        (Sales_df['Type'] == row['Type']) & 
        (Sales_df['IsHoliday'] == row['IsHoliday'])
    ].sort_values(by='Date')
    
    # Ensure Date is datetime and set as index
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
    filtered_df = filtered_df.set_index('Date')
    
    # Calculate rolling Standard Dev
    filtered_df['Rolling_Stdev'] = (
        filtered_df['Weekly_Sales']
        .rolling(window=window, min_periods=1)
        .std()
    )
    
    # Return the rolling STdev for the current row's date
    return filtered_df.loc[row['Date'], 'Rolling_Stdev']

## Called Modules to calculate Mean, StDev and 2 sigmna variation
Sales_df = Sales_df.sort_values(by=['Date', 'Type'])

Sales_df['RollingMean13Week'] = Sales_df.apply(
    calculate_rolling_mean_dynamic, 
    axis=1, 
    Sales_df=Sales_df, 
    window=13
)
"""

In [22]:
# Calculate the 52-week rolling mean for Type "A" and IsHoliday False
Sales_df['Rolling52WeekMean'] = (
    Sales_df.loc[Sales_df['Type'] == "A"]
    .loc[Sales_df['IsHoliday'] == False]
    .set_index('Date')['Weekly_Sales']
    .rolling(window=52, min_periods=1)
    .mean()
)

# Fill NaN for rows where the condition doesn't apply
Sales_df['Rolling52WeekMean'].fillna(0, inplace=True)

/tmp/ipykernel_7917/2500730219.py:2: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  Sales_df['Rolling52WeekMean'] = (


ValueError: cannot reindex on an axis with duplicate labels

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
